In [1]:
import pandas as pd
import numpy as np

# Cargar archivo
df = pd.read_excel("RBDSimce4toBasico2014-2024.xlsx")

print(df.shape)
df.head()


(7667, 62)


/Users/diegosarricolea/miniconda3/envs/vis/lib/python3.14/site-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


,agno,fecha_bbdd,codigo_bbdd,grado,rbd,dvrbd,nom_rbd,cod_reg_rbd,nom_reg_rbd,cod_pro_rbd,...,marca_lect4b_rbd,marca_mate4b_rbd,marca_soc4b_rbd,marcadif_lect4b_rbd,marcadif_mate4b_rbd,marcadif_soc4b_rbd,marca_eda_lect4b_rbd,marca_eda_mate4b_rbd,marca_eda_soc4b_rbd,noaplica
0,2014,20151009,final20151009v1,4b,5,1,JOVINA NARANJO FERNANDEZ,15,REGIÓN ARICA - PARINACOTA,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Rbd y curso rinde Simce
1,2014,20151009,final20151009v1,4b,8,6,COLEGIO INTEGRADO EDUARDO FREI MONTALVA,15,REGIÓN ARICA - PARINACOTA,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Rbd y curso rinde Simce
2,2014,20151009,final20151009v1,4b,9,4,ESCUELA REPUBLICA DE ISRAEL,15,REGIÓN ARICA - PARINACOTA,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Rbd y curso rinde Simce
3,2014,20151009,final20151009v1,4b,10,8,ESCUELA REPUBLICA DE FRANCIA,15,REGIÓN ARICA - PARINACOTA,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Rbd y curso rinde Simce
4,2014,20151009,final20151009v1,4b,11,6,ESC. GRAL. PEDRO LAGOS MARCHANT,15,REGIÓN ARICA - PARINACOTA,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Rbd y curso rinde Simce


In [2]:
cols = [
    "agno",
    "cod_com_rbd", "nom_com_rbd",
    "cod_reg_rbd", "nom_reg_rbd",
    "prom_lect4b_rbd", "prom_mate4b_rbd"
]

df = df[cols].copy()


In [3]:
df = df.dropna(subset=[
    "cod_com_rbd",
    "prom_lect4b_rbd",
    "prom_mate4b_rbd"
])


In [4]:
df["cod_com_rbd"] = df["cod_com_rbd"].astype(int)
df["agno"] = df["agno"].astype(int)
df["prom_lect4b_rbd"] = df["prom_lect4b_rbd"].astype(float)
df["prom_mate4b_rbd"] = df["prom_mate4b_rbd"].astype(float)


In [5]:
df_rm = df[df["cod_reg_rbd"] == 13].copy()

In [6]:
df_rm["nom_reg_rbd"].unique()

array(['REGIÓN METROPOLITANA'], dtype=object)

In [7]:
df_comunas = df_rm.groupby(
    ["cod_com_rbd", "nom_com_rbd"]
).agg({
    "prom_lect4b_rbd": "mean",
    "prom_mate4b_rbd": "mean"
}).reset_index()


In [8]:
import geopandas as gpd

comunas = gpd.read_file("comunas.shp")


In [10]:
comunas.columns

Index(['objectid', 'shape_leng', 'dis_elec', 'cir_sena', 'cod_comuna',
       'codregion', 'st_area_sh', 'st_length_', 'Region', 'Comuna',
       'Provincia', 'geometry'],
      dtype='object')

In [12]:
comunas_rm = comunas[comunas["codregion"] == 13].copy()

In [14]:
comunas_rm["COD_COM"] = comunas_rm["cod_comuna"].astype(int)


In [15]:
geo = comunas_rm.merge(
    df_comunas,
    left_on="cod_comuna",
    right_on="cod_com_rbd",
    how="left"
)


In [ ]:
import ipywidgets as widgets
import matplotlib.pyplot as plt

selector = widgets.Dropdown(
    options={
        "Lectura (promedio 2014–2024)": "prom_lect4b_rbd",
        "Matemática (promedio 2014–2024)": "prom_mate4b_rbd",
    },
    value="prom_lect4b_rbd",
    description="Asignatura:"
)


In [17]:
def plot_rm(columna):
    fig, ax = plt.subplots(figsize=(10, 10))

    geo.plot(
        column=columna,
        cmap="viridis",
        legend=True,
        edgecolor="black",
        linewidth=0.4,
        ax=ax
    )

    ax.set_title(
        f"SIMCE 4° Básico - Región Metropolitana\nPromedio histórico 2014–2024",
        fontsize=14
    )

    ax.axis("off")
    plt.show()


In [ ]:
widgets.interact(
    plot_rm,
    columna=selector
)


interactive(children=(Dropdown(description='Asignatura:', options={'Lectura (promedio 2014–2024)': 'prom_lect4…

<function __main__.plot_rm(columna)>